In [49]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pd.options.display.max_columns = None
pd.set_option("display.max_colwidth", -1)
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"
pd.set_option("display.max_rows", 101)

/home/nathaniel/anaconda3/envs/nba-predictor/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """


In [50]:
DATA_PATH = 'data'

SEASONS_PROCESSED_DS = f"{DATA_PATH}/seasons.processed.feather"

TEAMS_DS = f"{DATA_PATH}/teams.csv"
TEAMS_PROCESSED_DS = f"{DATA_PATH}/teams.processed.feather"

RANKING_DS = f"{DATA_PATH}/ranking.csv"
RANKING_PROCESSED_DS = f"{DATA_PATH}/ranking.processed.feather"

GAMES_DS = f"{DATA_PATH}/games.csv"
GAMES_PROCESSED_DS = f"{DATA_PATH}/games.processed.feather"

In [51]:
games = pd.read_csv(GAMES_DS, parse_dates=["GAME_DATE_EST"], usecols=["GAME_ID",'GAME_DATE_EST', 'GAME_STATUS_TEXT', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID',
       'SEASON', 'PTS_home', 'FG_PCT_home', 'FT_PCT_home',
       'FG3_PCT_home', 'AST_home', 'REB_home', 'PTS_away',
       'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away', 'REB_away',
       'HOME_TEAM_WINS']
                    , infer_datetime_format=True, index_col="GAME_ID")
games = games.sort_values(by=['GAME_DATE_EST', 'GAME_ID'])

teams = pd.read_feather(TEAMS_PROCESSED_DS)

seasons = pd.read_feather(SEASONS_PROCESSED_DS)

rankings = pd.read_csv(RANKING_DS, parse_dates=["STANDINGSDATE"], usecols=['TEAM_ID', 'LEAGUE_ID', 'SEASON_ID', 'STANDINGSDATE', 'CONFERENCE',
       'TEAM', 'G', 'W', 'L', 'W_PCT', 'HOME_RECORD', 'ROAD_RECORD'], infer_datetime_format=True,
                      index_col=["STANDINGSDATE"])
rankings.sort_index(inplace=True)

games_matchup = pd.read_feather(GAMES_PROCESSED_DS)
games_matchup = games_matchup.set_index(["GAME_ID"])
games_matchup = games_matchup.sort_values(by=['GAME_DATE_EST', 'GAME_ID'])

In [52]:
df = games_matchup
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4919 entries, 21500001 to 21801229
Data columns (total 70 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   GAME_DATE_EST          4919 non-null   datetime64[ns]
 1   HOME_TEAM_NAME         4919 non-null   object        
 2   HOME_TEAM_ID           4919 non-null   int64         
 3   VISITOR_TEAM_NAME      4919 non-null   object        
 4   VISITOR_TEAM_ID        4919 non-null   int64         
 5   GAME_STATUS_TEXT       4919 non-null   object        
 6   SEASON                 4919 non-null   int64         
 7   HT_RANK                4919 non-null   int64         
 8   HT_CLASS               4919 non-null   int64         
 9   HT_HW                  4919 non-null   int64         
 10  HT_HL                  4919 non-null   int64         
 11  HT_VW                  4919 non-null   int64         
 12  HT_VL                  4919 non-null   int64       

In [59]:
#df.loc[df.SEASON_ID > 20000, "SEASON_ID"] = df.loc[df.SEASON_ID > 20000, "SEASON_ID"] - 20000
#df.loc[df.SEASON_ID < 20000, "SEASON_ID"] = df.loc[df.SEASON_ID < 20000, "SEASON_ID"] - 10000
print(df.shape)
df[["HT_RANK", "HT_CLASS", "VT_RANK", "VT_CLASS", "HOME_TEAM_WINS"]].tail(30)

(4919, 70)


,HT_RANK,HT_CLASS,VT_RANK,VT_CLASS,HOME_TEAM_WINS
GAME_ID,,,,,
21801200,25,2,2,0,1
21801201,6,0,7,0,0
21801202,29,2,4,0,1
21801203,21,2,15,1,0
21801204,0,0,5,0,1
21801205,27,2,17,1,1
21801206,24,2,26,2,1
21801207,13,1,8,0,0
21801208,10,1,22,2,1


In [39]:
df.index.value_counts()

2018-07-17    30
2015-03-19    30
2007-10-10    30
2012-05-04    30
2016-11-27    30
              ..
2004-09-29    29
2004-01-05    29
2004-08-07    29
2004-03-23    29
2004-07-06    29
Name: STANDINGSDATE, Length: 5997, dtype: int64

In [47]:
df_game_day = df.loc[df.index == "2020-03-01"].sort_values("W_PCT")
df_game_day.reset_index(inplace=True)
rank = df_game_day[df_game_day.TEAM_ID == 1610612762].index[0]


21

In [41]:

#rankings_g = rankings[rankings.SEASON_ID >= 1015].groupby(["STANDINGSDATE"])
#rankings_g.describe()